In [2]:
!pip install pysindy numpy pyGPGO matplotlib scikit-learn scipy pathlib pandas cvxpy gurobipy
#installing all dependencies

  Using cached pysindy-1.7.5-py3-none-any.whl (121 kB)
  Using cached pyGPGO-0.5.1.tar.gz (14 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached cvxpy-1.4.1-cp39-cp39-win_amd64.whl (1.0 MB)
     --------------------------------------- 10.2/10.2 MB 16.4 MB/s eta 0:00:00
     ---------------------------------------- 8.4/8.4 MB 23.4 MB/s eta 0:00:00
  Using cached derivative-0.6.0-py3-none-any.whl (14 kB)
     --------------------------------------- 35.8/35.8 MB 27.3 MB/s eta 0:00:00
     -------------------------------------- 173.3/173.3 MB 2.4 MB/s eta 0:00:00
  Using cached Theano-PyMC-1.1.2.tar.gz (1.8 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached pymc3-3.11.5-py3-none-any.whl (872 kB)
  Using cached osqp-0.6.3-cp39-cp39-win_amd64.whl (292 kB)
  Using cached clarabel-0.6.0-cp37-abi3-win_amd64.whl (355 kB)
  Using cached pybind11-2.11.1-py3-no

ERROR: Cannot uninstall 'TBB'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


In [3]:
import numpy as np
import matplotlib.pyplot as plst
from mpl_toolkits import mplot3d
import pysindy as ps
import random
import pyGPGO
from pysindy import PolynomialLibrary, FourierLibrary 
from pyGPGO.surrogates.GaussianProcess import GaussianProcess
from pyGPGO.covfunc import squaredExponential
from pyGPGO.acquisition import Acquisition
from pyGPGO.GPGO import GPGO
from sklearn.metrics import mean_squared_error
#importing necessary modules

In [4]:
random_integers = [random.randint(1, 100000) for _ in range(1000)]

def err(param1, param2):

    #data_set = []
   # for it in range(1000):
   #     data_set.append(it)
    x = np.array(random_integers)
    time = np.linspace(0,1,1000)
    
    
#     model = ps.SINDy(feature_names=["x"])
#     model.fit(x,t = time)
#     x_derivative = model.differentiate(x)

    x_data = x
#    x_data_derivative = x_derivative
    
    differentiation_method = ps.FiniteDifference(order=2)
    
    poly_lib = ps.PolynomialLibrary(degree=int(param1))   
    trig_lib = ps.FourierLibrary(n_frequencies=int(param2)) 
    
    
    custom_lib = poly_lib + trig_lib
    feature_library = custom_lib
    
    optimizer = ps.STLSQ(threshold=0.2)
    
    model = ps.SINDy(
    differentiation_method=differentiation_method,
    feature_library=custom_lib,
    optimizer=optimizer,
    feature_names=["x"])
    model.fit(x_data,t = time)
    alpha = 1
    score = model.score(x_data,metric = mean_squared_error) # + alpha * complexity
    # ++ complexity
    
    return score

#defining the objective function which takes as inputs the degree and frequency of the libraries
#pentru fiecare pereche param1 si param2, functia err va calcula cu score eroarea dintre aproximarea facuta de model
#si derivata reala

In [5]:
#domeniu1 = np.arange(1, 101)
#domeniu2 = np.arange(1, 101)

cov = squaredExponential()
surogate = GaussianProcess(cov)
acq = Acquisition(mode = 'ExpectedImprovement')
params = {'param1' : ('int',[2,100]),
          'param2' : ('int',[2,100])}
np.random.seed(23)
gpgo = GPGO(surogate, acq, err,params)

gpgo.run(max_iter = 2)
print(gpgo.GP.y)



Evaluation 	 Proposed point 	  Current eval. 	 Best eval.
init   	 [85. 42.]. 	  21200090293.302906 	 21200090293.302906
init   	 [75. 56.]. 	  21200090293.302906 	 21200090293.302906
init   	 [33. 78.]. 	  21200090293.302906 	 21200090293.302906
1      	 [93. 41.]. 	  21200090293.302906 	 21200090293.302906
2      	 [56. 79.]. 	  21200090293.302906 	 21200090293.302906
[2.12000903e+10 2.12000903e+10 2.12000903e+10 2.12000903e+10
 2.12000903e+10]


In [7]:
"""import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

param1_values = np.arange(2, 101)
param2_values = np.arange(2, 101)

param1, param2 = np.meshgrid(param1_values, param2_values)

# Initialize an array to store the error values
errors = np.zeros_like(param1, dtype=float)

# Calculate the error for each combination of param1 and param2
for i in range(len(param1_values)):
    for j in range(len(param2_values)):
        errors[i, j] = err(param1_values[i], param2_values[j])

# Create a 3D plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(param1, param2, errors, cmap='viridis')

# Add labels and title
ax.set_xlabel('Ordin Polinom')
ax.set_ylabel('Ordin Trigonometric')
ax.set_zlabel('Error')
ax.set_title('Eroarea in 3D')

# Show the plot
plt.show()
""""

SyntaxError: EOL while scanning string literal (2960107327.py, line 31)